In [1]:
import sys

In [2]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [3]:
import numpy
import scipy
import pandas

In [4]:
import mysql_utils
import doc_proc
import init_tdm_tables

In [5]:
from importlib import reload

## Init Needed Tools

In [7]:
dd = lambda doc: doc_proc.build_text_feature(doc,
                                             components = ['title', 'summary'],
                                             lower=False,
                                             remove_stops=False,
                                             html_text=True,
                                            )

In [8]:
dp = init_tdm_tables.DocProcessor()

In [51]:
cnx = mysql_utils.getCnx()
cur = mysql_utils.getCur(cnx)

## Process

In [10]:
def query_wordIDLookup(cur, words):
    format_strings = ','.join(["%s"] * len(words))
    query = ("SELECT id, word FROM words "
             "WHERE word IN ({})".format(format_strings))
    cur.execute(query, list(words))
    cols = cur.column_names
    word_lookup = [{cols[0] : e[0], cols[1] : e[1]} for e in cur.fetchall()]
    word_lookup = {e['id'] : e['word'] for e in word_lookup}
    return(word_lookup)


def query_idWordLookup(cur, wids):
    format_strings = ','.join(["%s"] * len(wids))
    query = ("SELECT id, word FROM words "
             "WHERE id IN ({})".format(format_strings))
    cur.execute(query, list(wids))
    cols = cur.column_names
    word_lookup = [{cols[0] : e[0], cols[1] : e[1]} for e in cur.fetchall()]
    word_lookup = {e['id'] : e['word'] for e in word_lookup}
    return(word_lookup)

In [11]:
stops_lookup = query_wordIDLookup(cur, doc_proc.nltk_stops)

### Get Query Words

In [12]:
doc = {"title" : "FBI refused White House request to knock down recent Trump-Russia stories",
       "summary" : "Washington (CNN) The FBI rejected a recent White House request to publicly knock down media reports about communications between Donald Trump's associates and Russians known to US intelligence during the 2016 presidential campaign, multiple US officials briefed on the matter tell CNN."}

In [13]:
bow = dp.doc2BOW(dd(doc))

In [14]:
orig_query_words = [w for w in \
                    (query_wordIDLookup(cur,
                                       [w for w in bow if w not in doc_proc.nltk_stops])).keys()
                   ]

### Query for Docs that Share Words

In [15]:
def query_docsOnWords(cur, words, word_type="id", exclude_docs=set()):
    
    if word_type == "word":
        word_doc_query = ("SELECT doc_bows.doc_id, doc_bows.wcount "
                          "FROM  doc_bows LEFT JOIN words ON (doc_bows.word_id = words.id) "
                          "WHERE words.word = '{}' ")
    elif word_type == "id":
        word_doc_query = ("SELECT doc_bows.doc_id, doc_bows.wcount "
                          "FROM  doc_bows LEFT JOIN words ON (doc_bows.word_id = words.id) "
                          "WHERE words.id = '{}' ")

    doc_ids = set()
    word_count_store = []
    for word in words:
        cur.execute(word_doc_query.format(word))
        result = mysql_utils.dfDocsFromCursor(cur)
        result = result[[i not in exclude_docs for i in result['doc_id']]]
        if result.shape[0] > 0:
            doc_ids.update(set(result['doc_id']))
            word_count_store.append({'word' : word,
                                     'n_docs' : result.shape[0],
                                     'n_tot' : result['wcount'].sum()})

    doc_ids = [int(i) for i in doc_ids]
    word_count_store = pandas.DataFrame(word_count_store)
        
    return(doc_ids, word_count_store)


def word_summary_info(cur, words, wtype='id', exclude_docs=set(), include_docs=set()):
    """
    CREATE TABLE words_info SELECT doc_bows.word_id, 
    COUNT(doc_bows.doc_id) as n_docs, 
    SUM(doc_bows.wcount) as n_total  
    FROM  doc_bows 
    GROUP BY doc_bows.word_id;
    
    """
    
    if exclude_docs or include_docs:
        if exclude_docs:
            base = "AND doc_bows.doc_id NOT IN ({}) "
            dd = exclude_docs
        elif include_docs:
            base = "AND doc_bows.doc_id IN ({}) "
            dd = include_docs
        dd = [str(int(did)) for did in dd]
        dd = ", ".join(dd)
        ie_where_text = base.format(dd)
    else:
        ie_where_text = ""
        
        
    format_strings = ', '.join(['%s'] * len(words))
    
    if wtype=='word':
        where_clause = "WHERE words.word IN ({}) ".format(format_strings)
        where_clause += ie_where_text
        query = "SELECT doc_bows.word_id, COUNT(DISTINCT doc_bows.doc_id) as n_docs, " +\
                "SUM(doc_bows.wcount) as n_total " +\
                "FROM  doc_bows LEFT JOIN words ON (doc_bows.word_id = words.id) " +\
                where_clause +\
                "GROUP BY doc_bows.word_id"
        
    elif wtype=='id':
        where_clause = "WHERE doc_bows.word_id IN ({}) ".format(format_strings)
        where_clause += ie_where_text
        query = "SELECT doc_bows.word_id, COUNT(DISTINCT doc_bows.doc_id) as n_docs, " +\
                "SUM(doc_bows.wcount) as n_total " +\
                "FROM  doc_bows " +\
                where_clause +\
                "GROUP BY doc_bows.word_id"
            
    cur.execute(query, (words))
    result = mysql_utils.dfDocsFromCursor(cur)
    return(result)

#### Orig Doc

In [16]:
word_count_info = word_summary_info(cur, orig_query_words, wtype='id')

In [17]:
word_count_info.n_docs.describe()

count      31.000000
mean      723.741935
std      1476.303152
min        10.000000
25%        61.500000
50%       233.000000
75%       705.500000
max      7586.000000
Name: n_docs, dtype: float64

In [18]:
# Arbit...this should be "decayed"....see below
ndoc_cutoff = 100

qw_l01 = [int(w) for w in list(word_count_info[word_count_info.n_docs < ndoc_cutoff].word_id)]

In [19]:
# 2nd Pass: Get Doc IDs to use
docs_l01, wcs_l01 = query_docsOnWords(cur, qw_l01, word_type='id')

#### Level 1 Documents

In [61]:
def query_docBOW(cur, doc_id, word_list = []):
    query = "SELECT word_id, wcount FROM doc_bows WHERE doc_id = {}".format(doc_id)
    cur.execute(query)
    cols = cur.column_names
    bow = [{cols[0] : e[0], cols[1] : e[1]} for e in cur.fetchall()]
    if word_list:
        bow = [e for e in bow if e['word_id'] in word_list]
    bow = {e['word_id'] : e['wcount'] for e in bow}
    return(bow)

In [21]:
def query_AllDocWords(cur, doc_ids):
    doc_ids = list(doc_ids)
    format_strings = ','.join(['%s'] * len(doc_ids))
    query = ("SELECT DISTINCT word_id "
             "FROM doc_bows "
             "WHERE doc_id IN ({})".format(format_strings))
    cur.execute(query, (doc_ids))
    words = [e[0] for e in cur.fetchall()]
    return(words)

### This stepo is a bottleneck; can i pre-reduce the query words by using info from the sub-set?

In [22]:
def query_backgroundWordInfo(cur, word_ids):
    wids_format = ", ".join([str(wid) for wid in word_ids])
    query = "SELECT * FROM words_info WHERE word_id IN ({})".format(wids_format)
    cur.execute(query)
    result = mysql_utils.dfDocsFromCursor(cur)
    return(result)

Have the word stats for all words queued up already..in the "words_info" table

In [23]:
query_words = query_AllDocWords(cur, docs_l01)
query_words = [w for w in query_words if w not in stops_lookup.keys()]
len(query_words)

4122

In [26]:
n_docs_tot = mysql_utils.query_totalEntries(cur, 'rssfeed_links')
n_docs_tot

38317

In [27]:
bgwi = query_backgroundWordInfo(cur, query_words)
bgwi['frac'] = bgwi.n_docs / n_docs_tot

In [28]:
qwwi = word_summary_info(cur, query_words, wtype='id', include_docs=docs_l01)
qwwi['frac'] = qwwi.n_docs / len(docs_l01)

In [29]:
wi = bgwi.join(qwwi, on=['word_id'], lsuffix='_bg', rsuffix='_qw')

In [30]:
wi.head()

,word_id_bg,n_docs_bg,n_total_bg,frac_bg,word_id_qw,n_docs_qw,n_total_qw,frac_qw
0,1,105,144,0.002740,3.0,80.0,123,0.233918
1,3,7586,11670,0.197980,11.0,6.0,6,0.017544
2,7,91,93,0.002375,15.0,6.0,10,0.017544
3,11,803,932,0.020957,19.0,1.0,1,0.002924
4,12,310,382,0.008090,20.0,44.0,60,0.128655


In [31]:
wi['ratio'] = wi.frac_qw / wi.frac_bg

In [32]:
wi.ratio.describe()

count    1697.000000
mean        6.737447
std        20.801847
min         0.023449
25%         0.732275
50%         1.672209
75%         4.149556
max       336.114035
Name: ratio, dtype: float64

In [67]:
# Restrict query words based on "words_info" table in mysql
# Say, more than 5 docs, but less than 100 (50) (20) docs?
word_filter_l02 = lambda x: (x['n_docs_qw'] >= 3 and x['n_docs_bg'] < 75 and x['ratio'] > 2)

In [68]:
ids2ints = lambda id_series: [int(i) for i in list(id_series)]

In [69]:
qw_l02 = ids2ints(wi.word_id_bg[wi.apply(lambda entry: word_filter_l02(entry), axis=1)])
print(len(qw_l02))

112


In [70]:
docs_l02, wcs_l02 = query_docsOnWords(cur, qw_l02,
                                      word_type="id", exclude_docs=docs_l01)

OperationalError: 2055: Lost connection to MySQL server at '127.0.0.1:3306', system error: 9 Bad file descriptor

In [37]:
len(docs_l02)

1732

In [38]:
qww = query_idWordLookup(cur, qw_l02)

## Generate Network from Selected Vocab, Docs

In [39]:
words = set(orig_query_words).copy()
words.update(qw_l01)
words.update(qw_l02)
len(words)

106

In [40]:
for w in words:
    print(w)

3072
1793
3
1539
1540
3080
1040
22
535
1302
1561
1563
284
12578
290
292
2853
301
302
2863
1585
1586
5427
2100
2613
4148
311
56
6202
826
3134
27973
1095
2632
1354
1355
2635
1869
2890
1618
339
2900
2901
2645
855
1625
2142
607
360
2153
1131
1648
113
1398
121
2938
124
3453
1153
898
902
1159
1670
2702
400
147
3220
410
155
156
412
668
1693
1694
1193
683
1196
3757
432
1968
2481
1716
6327
1207
955
1725
3262
1473
455
1741
1234
2775
218
987
1754
2268
222
990
2272
3034
2020
744
1001
241
2294
2297


In [54]:
orig_bow_lookup = {w : i for i,w in query_wordIDLookup(cur, bow.keys()).items()}

In [56]:
bow = {orig_bow_lookup[k] : wcount for k,wcount in bow.items()}

In [62]:
bows = [{'doc_id' : 'orig', 'bow' : bow}]
for d in docs_l01:
    bows.append({'doc_id' : d,
                 'bow' : query_docBOW(d, word_list=words)})
for d in docs_l02:
    bows.append({'doc_id' : d,
                 'bow' : query_docBOW(d, word_list=words)})
print(len(bows))

2075


In [65]:
cnx.close()

In [47]:
import pickle

In [48]:
bows_path = os.path.join(os.path.abspath(os.path.join('../..')), 'src/data/sample_bows.pkl')

In [66]:
with open(bows_path, 'wb') as f:
    pickle.dump(bows, f)

In [1]:
from importlib import reload

In [ ]:
reload(mysql_ut)